# mzML and psm

Some useful functions:
* load psm data and sort it 
* mzML helper function
* join data function
* save file function
* controller function

In [138]:
import pandas as pd
import numpy as np
import os
import pyteomics.mzml
import spectrum_utils.spectrum as sus
from pathlib import Path
import json

In [169]:
def load_psm(psm_file_path):
    # read the psm file into a pandas dataframe
    psm_dataframe = pd.read_table(psm_file_path, delimiter='\t')

    # sort dataframe by QValue
    psm_dataframe = psm_dataframe.sort_values("QValue")

    # drop duplicates
    psm_dataframe = psm_dataframe.drop_duplicates(subset=["Scan Number"], keep="first")
    psm_dataframe["Protein Accession"] = psm_dataframe["Protein Accession"].astype(str)

    # rename the "Full Sequence" column
    psm_dataframe = psm_dataframe.rename({"Full Sequence": "Peptide"})

    return psm_dataframe

def load_psm_df_msfragger(psm_file_path):
    # read in the psm file as a dataframe
    psm_df = pd.read_table(psm_file_path)

    # split the "Spectrum" column into a list at each period and store it under 
    # the "temp_split_column"
    psm_df["temp_split_column"] = psm_df["Spectrum"].str.split(".")
    # store the element located at index 1 of the "temp_split_column" in a 
    # "Scan Number" column
    psm_df["Scan Number"] = psm_df["temp_split_column"].map(lambda x:x[1]).apply(pd.to_numeric)
    # drop unneeded columns
    columns_to_drop = ['Spectrum', 'temp_split_column','Mapped Genes', 'Mapped Proteins']
    psm_df = psm_df.drop(columns=columns_to_drop)

    # rename 2 headers to match MM file formats
    psm_df = psm_df.rename({'Protein ID': 'Protein Accession', 'Spectrum File': 'File Name'}, axis=1)

    # drop duplicates
    psm_df = psm_df.drop_duplicates(subset=["Scan Number"], keep="first")
    
    return psm_df

In [170]:
def load_mzml_dict(mzml_file_path):
    # use pyteomics.mzml.read() to generate an iterator over the dicts with spectrum properties
    mzml_dicts = pyteomics.mzml.read(source=mzml_file_path)
    return mzml_dicts

psm and peptideQ (for total as well) and mzml_psm
['File Name','Protein Accession','Peptide', 'Scan Number']

peptideQ and protein
['Protein Accession', 'Peptide']

mzml
['Scan Number']

protein
['Protein Accession']


In [171]:
def load_mzml_df(mzml_dicts):
    # load dataframe from the list of mzml dictionaires
    # drop the extra index column
    mzml_df = pd.DataFrame(mzml_dicts).drop(columns='index')

    # create a new dataframe containing only the ms/ms scans
    relevant_info = mzml_df.loc[(mzml_df['ms level'] == 2)]

    # reset the index to make up for the ms scans that were not included in this database
    relevant_info = relevant_info.reset_index(drop=True)

    # drop irrelevent columns (Note: We can change this if needed.)
    columns_to_drop = ["spectrum title", "count", "positive scan", "centroid spectrum", "defaultArrayLength", "MSn spectrum", "dataProcessingRef", "scanList", "MS1 spectrum", "ms level"]
    relevant_info = relevant_info.drop(columns=columns_to_drop)

    # create a new "Scan Number" column
    # the scan number info is contained within the "id" column so we will pull out the scan number and then delete the "id" column
    relevant_info["temp_split_column"] = (relevant_info["id"].str.split(" "))
    relevant_info["Scan Number"] = relevant_info["temp_split_column"].map(lambda x:x[2]).str.replace("scan=", "")
    relevant_info["Scan Number"] = relevant_info["Scan Number"].apply(pd.to_numeric)
    relevant_info = relevant_info.drop(columns=['temp_split_column', 'id'])

    # next, we'll want to pull out some info about the precursor in the "precursorList" columm
    # we will store the info we want under the "precursor info" column
    # then we'll drop the "precursorList" column
    relevant_info["precursor info"] = relevant_info["precursorList"].map(lambda x:x['precursor'][0]['selectedIonList']['selectedIon'][0]).astype(str)
    relevant_info = relevant_info.drop(columns=["precursorList"])

    # the precursor info is stored as a string in the format of a dictionary
    # json.loads() requires " instead of ' so we will fix that then convert the string into a dictionary
    dict_list = relevant_info["precursor info"].tolist()

    for index, dictionary in enumerate(dict_list):
        dictionary = dictionary.replace("'", '"')
        dict_list[index] = json.loads(dictionary)

    # we'll then load the dictionary data into a temporary dataframe
    three_column_df = pd.DataFrame.from_dict(dict_list)

    # next, we'll concatenate these two dataframes along the columns based on the index
    complete_mzml_df = pd.concat([relevant_info, three_column_df], axis="columns")

    # drop the "precursor info" column because we don't need it anymore
    complete_mzml_df = complete_mzml_df.drop(columns=['precursor info'])

    # as the scan number is the index we care about, we will use it as our index in the dataframe
    complete_mzml_df= complete_mzml_df.set_index("Scan Number")

    return complete_mzml_df 



In [172]:
# save the joined dataframe as a tsv file

def save_df(joined_dataframe, file_path):
    joined_dataframe.to_csv(file_path, sep="\t", index=False)
    print(f'Dataframe saved.')

In [184]:
def mzml_and_psm_controller(file_type, mzml_file_path,psm_file_path, mzml_and_psm_file_path, columns_to_keep=None):

    ''' Joins an mzml and psm file. Files are joined into a pandas dataframe and saved as a tsv.
    
    Required Parameters:
        * file_type: "mm" for metamorpheus files, or "msfragger" for msfragger files
        * mzml_file_path: File path to the mzML file
        * psm_file_path: File path to the psm file
        * mzml_and_psm_file_path: Output file path
        
    Optional Parameters: 
        * columns_to_keep: List of columns to include in the dataframe. Note that column names may vary based on whether your files were generated with MetaMorpheus or MSFragger.'''

    # load psm dataframe based on psm file type
    if file_type.lower() == 'mm':
        psm_dataframe = load_psm(psm_file_path)
    elif file_type.lower() == 'msfragger':
        psm_dataframe = load_psm_df_msfragger(psm_file_path=psm_file_path)
    else:
        print('invalid file type')
        return 
    
    # load mzML dataframe
    list_of_mzml_dicts = load_mzml_dict(mzml_file_path=mzml_file_path)
    mzml_dataframe = load_mzml_df(mzml_dicts=list_of_mzml_dicts)

    # merge datafames based on "Scan Number"
    joined_dataframe = mzml_dataframe.join(other=psm_dataframe, on='Scan Number', how='inner')

    # select all columns to keep, if this parameter was not passed in, return dataframe with all columns
    if columns_to_keep != None:
        joined_dataframe = joined_dataframe[columns_to_keep]

    # set up the multiIndex
    joined_dataframe = joined_dataframe.set_index(['File Name', 'Protein Accession', 'Peptide', 'Scan Number'])

    # save merged dataframes
    save_df(joined_dataframe=joined_dataframe, file_path=mzml_and_psm_file_path)

    return joined_dataframe

In [185]:
help(mzml_and_psm_controller)

Help on function mzml_and_psm_controller in module __main__:

mzml_and_psm_controller(file_type, mzml_file_path, psm_file_path, mzml_and_psm_file_path)
    Joins an mzml and a psm file. Files are joined into a pandas dataframe and saved as a tsv.
    
    Required Parameters:
        * file_type: "mm" for metamorpheus files, or "msfragger" for msfragger files
        * mzml_file_path: File path to the mzML file
        * psm_file_path: File path to the psm file
        * mzml_and_psm_file_path: Output file path
        
    Optional Parameters: 
        * columns_to_keep: List of columns to include in the dataframe. Note that column names may vary based on whether your files were generated with MetaMorpheus or MSFragger.



In [174]:
def short_controller_function():

    psm_dataframe = load_psm(psm_file_path)
    # merge datafames
    joined_dataframe = relevant_info.join(other=psm_dataframe, on="Scan Number", how='inner', lsuffix='_mzml', rsuffix='_psm')
    joined_dataframe = joined_dataframe.drop(columns=['Scan Number_mzml'])
    joined_dataframe = joined_dataframe.set_index(['File Name', 'Protein Accession'])

    # save merged dataframes
    save_df(joined_dataframe=joined_dataframe, file_path=output_file_path)

    return joined_dataframe

In [175]:
mzml_file_path = "C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\MetaM\\2ng\\Ex_Auto_J3_30umTB_2ngQC_60m_1.mzML"
psm_file_path = "C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\msfragger\\psm1.tsv"
output_file_path = "C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\MetaM\\2ng\\mzml_and_psm.tsv"

In [176]:
# testing the function
new_database = (file_type='msfragger', psm_file_path=psm_file_path, mzml_and_psm_file_path=output_file_path)

Dataframe saved.


In [179]:
new_database

base peak m/z  \
File Name                                       Protein Accession Peptide         Scan Number                  
interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml P18206            QQELTHQEHR      4761           3045.887847   
                                                P18077            NNTVTPGGKPNK    4770            226.334431   
                                                P67809            ENQGDETQGQQPPQR 4772           3684.796666   
                                                P11413            PASTNSDDVRDEK   4777           1438.513781   
                                                Q14980            AVQAQGGESQQEAQR 4779            355.070984   
...                                                                                                      ...   
                                                P46782            TIAECLADELINAAK 20457          1018.494557   
                                                O43175            DLPLLLFR        20487           251.103354   
                                                P08195            LLTSFLPAQLLR    20534           129.102685   
                                                P13639            TFCQLILDPIFK    20541           427.747145   
                                                P46783            IAIYELLFK       20783           451.267751   

                                                                                               base peak intensity  \
File Name                                       Protein Accession Peptide         Scan Number                        
interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml P18206            QQELTHQEHR      4761                 1042.273804   
                                                P18077            NNTVTPGGKPNK    4770                 1360.701294   
                                                P67809            ENQGDETQGQQPPQR 4772                 1703.962402   
                                                P11413            PASTNSDDVRDEK   4777                  823.853760   
                                                Q14980            AVQAQGGESQQEAQR 4779                 2075.281982   
...                                                                                                            ...   
                                                P46782            TIAECLADELINAAK 20457               72908.125000   
                                                O43175            DLPLLLFR        20487               45406.714844   
                                                P08195            LLTSFLPAQLLR    20534               20830.025391   
                                                P13639            TFCQLILDPIFK    20541              107592.500000   
                                                P46783            IAIYELLFK       20783               39816.992188   

                                                                                               total ion current  \
File Name                                       Protein Accession Peptide         Scan Number                      
interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml P18206            QQELTHQEHR      4761              11691.656250   
                                                P18077            NNTVTPGGKPNK    4770              14130.234375   
                                                P67809            ENQGDETQGQQPPQR 4772              17414.609375   
                                                P11413            PASTNSDDVRDEK   4777              12231.937500   
                                                Q14980            AVQAQGGESQQEAQR 4779              24020.359375   
...                                                                                                          ...   
                                                P46782            TIAECLADELINAAK 20457            430894.437500   
                                                O43175     

### Using this code, lets do the mzml and protein

In [134]:
def join_protein_to_df(protein_df, mzml_psm_df):
    # here we'll delete the duplicate columns in the protein_df
    duplicate_columns = []
    for column in protein_df.columns:
        if column in mzml_psm_df.columns:
            duplicate_columns.append(column)
    protein_df=protein_df.drop(axis=1,columns=duplicate_columns)
    new_df = mzml_psm_df.join(other=protein_df, on="Protein Accession", how='left', lsuffix="_psm")
    return new_df


def mzml_and_protein_controller(mzml_file_path, psm_file_path, protein_file_path, output_file_path, all_data=False):
    # load dataframes
    psm_dataframe = load_psm(psm_file_path)
    protein_dataframe = pd.read_table(protein_file_path)

    count = 0
    for index, row in psm_dataframe.iterrows():
        # break put here to faciliate testing
        # remove this conditional statement when parsing through an entire mzml file
        if count == 2:
            break
        print(f"{count} : {index}")
        scan_num = str(row['Scan Number'])
        relevant_info = mzml_helper(scan_num, mzml_file_path, all_data)
        psm_dataframe = join_data(relevant_info, psm_dataframe, index)
        count += 1
    
    # join dataframes and save as a tsv
    joined_df = join_protein_to_df(protein_df=protein_dataframe, mzml_psm_df=psm_dataframe)
    save_file(psm_dataframe=joined_df, file_path=output_file_path)
    
    return joined_df

In [135]:
# test
protein_file_path = "C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\MetaM\\02ng\\AllQuantifiedProteinGroups.tsv"
outfile_path = "C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\MetaM\\02ng\\mzml_and_protein.tsv"
test_df = mzml_and_protein_controller(mzml_file_path=mzml_file_path, psm_file_path=psm_file_path, protein_file_path=protein_file_path, output_file_path=outfile_path)


0 : 0


NameError: name 'mzml_helper' is not defined

In [ ]:
test_df.head()


,File Name,Scan Number,Scan Retention Time,Num Experimental Peaks,Total Ion Current,Precursor Scan Number,Precursor Charge,Precursor MZ,Precursor Mass,Score,...,Intensity_Ex_Auto_K13_30umTA_2ngQC_60m_2-calib,Intensity_Ex_Auto_W17_30umTB_2ngQC_60m_1-calib,Intensity_Ex_Auto_W17_30umTB_2ngQC_60m_2-calib,Number of PSMs,Protein Decoy/Contaminant/Target,Protein Cumulative Target,Protein Cumulative Decoy,Protein QValue,Best Peptide Score,Best Peptide Notch QValue
0,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,16668,52.52059,101.0,335251.91071,16649,2.0,1280.62784,2559.24113,28.510,...,1.935038e+07,1.823499e+07,355.0,T,2.0,0.0,0.0,29.506938,0.0,NaN
1227,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,20217,60.99236,49.0,76071.94836,20203,3.0,800.73355,2399.17882,13.388,...,1.413019e+06,1.530572e+06,62.0,T,45.0,0.0,0.0,24.292929,0.0,NaN
1226,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,21843,64.88158,77.0,185690.32166,21831,3.0,515.26251,1542.76569,13.388,...,1.662131e+07,1.475907e+07,462.0,T,129.0,0.0,0.0,21.589114,0.0,NaN
1225,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,16915,53.10976,99.0,224170.22906,16902,3.0,506.92071,1517.74029,13.390,...,3.405293e+06,3.366013e+06,395.0,T,27.0,0.0,0.0,25.311353,0.0,NaN
1224,Ex_Auto_J3_30umTB_2ngQC_60m_1-calib,19666,59.67404,72.0,282899.35223,19653,2.0,721.90597,1441.79739,13.390,...,5.032313e+06,4.587147e+06,109.0,T,286.0,0.0,0.0,19.439414,0.0,NaN
